<!--<img width=700px; src="../img/logoUPSayPlusCDS_990.png"> -->

<p style="margin-top: 3em; margin-bottom: 2em;"><b><big><big><big><big>Introducción a Pandas</big></big></big></big></b></p>

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_rows = 8

# 1. Iniciemos con un Ejemplo

#### Caso 1: datos de sobrevivientes del Titanic

In [ ]:
df = pd.read_csv("data/titanic.csv")

In [ ]:
df.head()

De leer los datos, a contestar preguntas con tan solo unas lineas de código:

**¿Cual es la distribución de edad de los pasajeros?**

In [ ]:
df['Age'].hist()

**¿Cómo difiere la proporción de supervivencia entre hombres y mujeres?**

In [ ]:
df.groupby('Sex')[['Survived']].aggregate(lambda x: x.sum() / len(x))

**¿O cómo difiere entre clases?**

In [ ]:
df.groupby('Pclass')['Survived'].aggregate(lambda x: x.sum() / len(x)).plot(kind='bar')

Toda la funcionalidad requerida para entender estos ejemplos será explicada a través de este tutorial.

#### Caso 2: Series de tiempo de medición de calidad de aire

AirBase (La base de datos de calidad de aire Europea): Mediciones cada hora de calidad de aire en estaciones de monitoreo ubicadas en Europa

Empezando con estos datos de cada hora:

In [ ]:
data = pd.read_csv('data/20000101_20161231-NO2.csv', sep=';', skiprows=[1], na_values=['n/d'], index_col=0, parse_dates=True)

In [ ]:
data.head()

A solucionar problemas de obtención de datos:

**¿Hay un decremento general en la contaminación en el aire?**

In [ ]:
data['1999':].resample('M').mean().plot(ylim=[0,120])

In [ ]:
data['1999':].resample('A').mean().plot(ylim=[0,100])

**¿Cual es la diferencia entre el perfil diurno de calidad de aire para días entre semana y de fin de semana?

In [ ]:
data['weekday'] = data.index.weekday
data['weekend'] = data['weekday'].isin([5, 6])
data_weekend = data.groupby(['weekend', data.index.hour])['BASCH'].mean().unstack(level=0)
data_weekend.plot()

Volveremos a estos ejemplos para construirlos paso por paso.

# 2. Pandas: Análisis de Datos en Python

Para trabajo de datos intensivo en Python, la libreria [Pandas](http://pandas.pydata.org) se ha vuelto esencial.

¿Qué es `pandas`?

* Pandas puede verse como una versión de arreglos de NumPy con etiquetas para filas y columnas, y mejor soporte para tipos heterogeneos de datos, pero también es mucho más que eso.
* Pandas también puede verse como el tipo `data.frame` de `R`, implementado en Python.
* Muy útil para trabajar con datos que hagan falta, series de tiempo, lectura y escritura de datos, modificacion estructural (reshaping), agrupación, mezcla de datos, etc...

Su documentación: http://pandas.pydata.org/pandas-docs/stable/

** ¿Cuando necesitamos pandas? **

Cuando se trabaja con **datos estructurados o tabulados** (Como un dataframe de R, una tabla SQL, una hoja de Excel, ...):

- Importar datos
- Limpiar datos sucios/desordenados
- Explorar sets de datos, ganar insight
- Procesar y preparar datos para análisis posterior
- Analizar datos (usando pandas junto con scikit-learn, statsmodels, etc)

<div class="alert alert-warning">
<b>ATENCIÓN!</b>: <br><br>

Pandas es bueno para trabajar con datos tabulados y heterogeneos de datos en 1 o 2 dimensiones, pero no todos los tipos de datos puede ser representados por estas estructuras
<ul>
<li>Cuando se trabajan con datos multidimensionales de un solo tipo, es mejor usar numpy.</li>
<li>Cuando se utilizan datos multidimensionales (datos climáticos): usar [xarray](http://xarray.pydata.org/en/stable/)</li>
</ul>
</div>

# 2. Las estructuras de Datos de Pandas: `DataFrame` y `Series`

Un `DataFrame` es una **estructura de datos tabulada** (objeto multi-dimensional para almacenar datos etiquetados), compuesta de filas y columnas, similar a una hoja de Excel o una tabla de base de datos. Puede pensarse en dicha estructura como múltiples objetos de tipo Series que comparten el mismo índice.

<img align="left" width=50% src="img/schema-dataframe.svg">

In [ ]:
df

### Atributos del DataFrame

Un DataFrame, aparte de tener un atributo de `index`, posee también un atributo `columns`:

In [ ]:
df.index

In [ ]:
df.columns

Para ver el tipo de datos de cada columna:

In [ ]:
df.dtypes

Una vista general de la información contenida puede ser obtenida usando el método `info()`:

In [ ]:
df.info()

Un DataFrame contiene también un atributo de `values`. Al trabajar con datos heterogeneos, todos los valores van a estar en mayúsculas.

In [ ]:
df.values

Aparte de importar datos de una fuente externa (archivo de texto, excel, base de datos), una de las maneras más comunes de crear un DataFrame es a partir de un diccionario o un arreglo de listas.

Nótese que en el cuaderno de IPython, el DataFrame siempre se va a mostrar como una tabla HTML:

In [ ]:
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
df_countries = pd.DataFrame(data)
df_countries

### Datos de una Dimensión: `Series` (la columna de DataFrame)

Un objeto Series es un contenedor para **datos etiquetados de una dimensión**.

In [ ]:
df['Age']

In [ ]:
age = df['Age']

### Atributos del objeto Series: `index` y `values`

El objeto Series también tiene un atributo `index` y `values`, pero no el de `columns`.

In [ ]:
age.index

Puede accederse a la representación de los datos en forma de arreglo de numpy usando el atributo `.values`.

In [ ]:
age.values[:10]

Se accede al valor por medio de un índice, de manera similar a los arreglos de numpy.

In [ ]:
age[0]

A diferencia de NumPy, este índice puede ser algo más que un entero.

In [ ]:
df = df.set_index('Name')
df

In [ ]:
age = df['Age']
age

In [ ]:
age['Dooley, Mr. Patrick']

Debido a que el DataFrame se basa en arreglos de NumPy, es posible aplicar muchos de los principios y operaciones aplicables a estos a los objetos de DataFrame y Series.

Ej operaciones por elemento:

In [ ]:
age * 1000

Un rango de métodos:

In [ ]:
age.mean()

Indexación avanzada (usando listas de indices, o condiciones booleanas):

In [ ]:
age[age > 70]

Así como métodos exclusivos a Pandas:

In [ ]:
df['Embarked'].value_counts()

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>¿Cuál es el valor máximo y la media de los precios pagados por tiquetes entre supervivientes del titanic?</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion31.py

In [ ]:
# %load snippets/01 - Introduccion32.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Calculate la proporción de supervivencia promedio para todos los pasajeros (nota: la columna 'Survived' indica si alguien sobrevivió (1) o no (0)).</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion33.py

# 3. Importación y Exportación de Datos

Pandas soporta muchos formatos de entrada para datos de forma nativa:

* CSV, text
* SQL database
* Excel
* HDF5
* json
* html
* pickle
* sas, stata
* (parquet)
* ...

In [ ]:
#pd.read

In [ ]:
#df.to

Muy flexible para leer archivos csv:

In [ ]:
pd.read_csv?

Afortunadamente, para un archivo bien formateado, no vamos a ocupar muchas de estas opciones.

In [ ]:
df = pd.read_csv("data/titanic.csv")

In [ ]:
df.head()

<div class="alert alert-success">

<b>EJERCICIO</b>: Leer el archivo `data/20000101_20161231-NO2.csv` a un DataFrame llamado `no2`
<br><br>
Algunos aspectos de este archivo a considerar:
 <ul>
  <li>¿Cuál es el separador usado en el archivo?</li>
  <li>La segunda fila indica información unitaria y debe ser ignorada (revisar `skiprows`)</li>
  <li>Para datos faltantes, utiliza la notación `'n/d'`. (revisar `na_values`)</li>
  <li>Queremos interpretar la columna `timestamp` como datetimes (revisar `parse_dates`)</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion39.py

In [ ]:
no2

# 4. Exploración

Dos métodos útiles:

`head` y `tail`

In [ ]:
no2.head(3)

In [ ]:
no2.tail()

`info()`

In [ ]:
no2.info()

Obtener datos y estadísticas básicas usando el método `describe`:

In [ ]:
no2.describe()

Visualización rápida de datos:

In [ ]:
no2.plot(kind='box', ylim=[0,250])

In [ ]:
no2['BASCH'].plot(kind='hist', bins=50)

<div class="alert alert-success">

<b>EJERCICIO</b>: 

 <ul>
  <li>Grafique la distribución de edad para los pasajeros del Titanic</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion47.py

La graficación por defecto (al no especificar `kind`) es un grafico de lineas para todas las columnas:

In [ ]:
no2.plot(figsize=(12,6))

Esto no nos dice mucho..

Podemos seleccionar solamente una porción de los datos (ej: solo las últimas 500 muestras en el dataset):

In [ ]:
no2[-500:].plot(figsize=(12,6))

O podemos utilizar funciones más avanzadas de series temporales (ver más adelante!)

# 5. Selección y Filtración de Datos

<div class="alert alert-warning">
<b>ATENCIÓN!</b>: <br><br>

Una de las funciones básicas de pandas es el aplicar etiquetas a filas y columnas, pero esto complica la indexación al compararse con Numpy. Ahora tenemos que distinguit entre:

 <ul>
  <li>Selección por **etiqueta**</li>
  <li>Selección por **posición**</li>
</ul>
</div>

In [ ]:
df = pd.read_csv("data/titanic.csv")

### `df[]` Provee algunos atajos convenientes.

Para un DataFrame, indexación básica selecciona columnas.

Seleccionar una única columna:

In [ ]:
df['Age']

o múltiples columnas:

In [ ]:
df[['Age', 'Fare']]

Al utilizar Slicing, se selecciona entre filas

In [ ]:
df[10:15]

### Indexación Sistemática usando `loc` y `iloc`

Al utilizar `[]` como arriba, se selecciona de más de un eje a la ves (filas o columnas, no ambas). Para indexación avanzada, se tienen los atributos extras:

* `loc`: Selección por etiqueta.
* `iloc`: Selección por posición.

In [ ]:
df = df.set_index('Name')

In [ ]:
df.loc['Bonnell, Miss. Elizabeth', 'Fare']

In [ ]:
df.loc['Bonnell, Miss. Elizabeth':'Andersson, Mr. Anders Johan', :]

Seleccionar por posición con `iloc` funciona de manera similar a indexar con arreglos de Numpy:

In [ ]:
df.iloc[0:2,1:3]

Los diferentes métodos de indexación pueden usarse también para asignar datos:

In [ ]:
df.loc['Braund, Mr. Owen Harris', 'Survived'] = 100

In [ ]:
df

### Indexación Booleana (Filtros)

A menudo, se deseará seleccionar filas basadas en ciertas condiciones. Esto puede lograrse usando `indexación booleana`, similar a una cláusula Where en SQL.

El indexador (la máscara booleana) debe ser de una dimensión y de la misma longitud que lo que se está indexando.

In [ ]:
df['Fare'] > 50

In [ ]:
df[df['Fare'] > 50]

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Basándose en el data set del titanic, seleccione todas las filas de pasajeros hombres y calcule la edad promedio de dichos pasajeros. Haga lo mismo para los pasajeros del género opuesto.</li>
</ul>
</div>

In [ ]:
df = pd.read_csv("data/titanic.csv")

In [ ]:
# %load snippets/01 - Introduccion63.py

In [ ]:
# %load snippets/01 - Introduccion64.py

In [ ]:
# %load snippets/01 - Introduccion65.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Basándose en los datos del Titanic, ¿Cuantos pasajeros habían con una edad superior a 70?</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion66.py

In [ ]:
# %load snippets/01 - Introduccion67.py

# 6. La Operación GroupBy

### Un poco de teoría: la operación groupby (split-apply-combine)

In [ ]:
df = pd.DataFrame({'key':['A','B','C','A','B','C','A','B','C'],
                   'data': [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

### Repaso: Funciones de agregación

Al analizar datos, es común calcular estadísticas de resumen (agregaciones como la media, el promedio, valor máximo...). Como hemos visto, es sencillo calcular estas estadísticas para series o columnas usando los métodos disponibles. Por ejemplo:

In [ ]:
df['data'].sum()

Sin embargo, en muchos casos los datos tienen ciertos grupos dentro de ellos, y en este caso, es posible que se desee calcular esta estadística para cada uno de estos grupos.

Por ejemplo, en el dataframe `df`, existe una columna 'key' con tres posibles valores: 'A', 'B' y 'C'. Cuando queremos cualcular la suma para cada uno de estos grupos, podríamos ejecutar lo siguiente:

In [ ]:
for key in ['A', 'B', 'C']:
    print(key, df[df['key'] == key]['data'].sum())

Esto se vuelve complicado al tener varios grupos. Podemos facilitar esta operación generando un ciclo que evalue lo mismo para los diferentes valores, pero sigue siendo poco conveniente.

Lo aplicado arriba, una misma operación a diferentes grupos, es una operación groupby, y pandas provee soporte para funciones de conveniencia que facilitan su aplicación.

### Groupby: Aplicando funciones por grupo

El concepto de "group by": queremos **aplicar la misma función a subconjuntos del dataframa, basándose en una llave para dividir el dataset en subconjuntos**.

Esta operación también se refiere a "split-apply-combine", involucrando los siguientes pasos:

* **División** de los datos basándose en un criterio determinado.
* **Apicación** de una función a cada grupo.
* **Combinación** de los resultados en una única estructura.

<img src="img/splitApplyCombine.png">

Similar a la operación SQL `GROUP BY`

En vez de generar un filtro manual:

    df[df['key'] == "A"].sum()
    df[df['key'] == "B"].sum()
    ...

Pandas provee el método `groupby` para hacer esto:

In [ ]:
df.groupby('key').sum()

In [ ]:
df.groupby('key').aggregate(np.sum)  # 'sum'

Y muchos otros métodos disponibles

In [ ]:
df.groupby('key')['data'].sum()

### Aplicación del concepto groupby sobre datos del Titanic.

Volvemos al set de datos acerca de sobrevivientes del Titanic:

In [ ]:
df = pd.read_csv("data/titanic.csv")

In [ ]:
df.head()

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Calcular el promedio de edad para cada género, ahora usando groupby.</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion76.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Calcular la proporción de supervivencia para todos los pasajeros.</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion77.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Calcular la proporción de sobrevivientes para menores de 25 años (recordar el uso de indexación booleana).</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion78.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>¿Cuál es la diferencia entre la proporción de supervivencia entre ambos géneros?</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion79.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>¿Cómo difiere entre clases? Utilize un gráfico de barras para visualizar la proporción de supervivencia para las 3 clases.</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion80.py

<div class="alert alert-success">

<b>EJERCICIO</b>:

 <ul>
  <li>Elabore un gráfico de barras que visualize el precio de tiquete promedio por personas dependiendo de su edad. La columna de edad es separada usando el método `pd.cut` provisto a continuación.</li>
</ul>
</div>

In [ ]:
df['AgeClass'] = pd.cut(df['Age'], bins=np.arange(0,90,10))

In [ ]:
# %load snippets/01 - Introduccion82.py

# 7. Trabajando con datos temporales

In [ ]:
no2 = pd.read_csv('data/20000101_20161231-NO2.csv', sep=';', skiprows=[1], na_values=['n/d'], index_col=0, parse_dates=True)

Cuando un DataFrame contiene un `DatatimeIndex`, las funcionalidades relacionadas a series temporales se vuelven disponibles:

In [ ]:
no2.index

Indexando una serie de tiempo usando Strings:

In [ ]:
no2["2010-01-01 09:00": "2010-01-01 12:00"]

Pandas también provee soporte para indexación parcial por String, por lo que no es necesario proveer un String completo:

Ej. Todos los datos desde enero hasta marzo del 2012:

In [ ]:
no2['2012-01':'2012-03']

Componentes de hora y fecha pueden ser accedidos desde el `index`:

In [ ]:
no2.index.hour

In [ ]:
no2.index.year

## Convirtiendo Series de Tiempo con `resample`

**`resample`: convertir la frecuencia en una serie de tiempo** es un método flexible y poderoso (por ejemplo, convertir datos de frecuencia horaria a diaria).

Recordemos el ejemplo de calidad de aire:

In [ ]:
no2.plot()

La serie de tiempo tiene una frecuencia de 1 hora. Queremos modificar esto a que sea diario:

In [ ]:
no2.head()

In [ ]:
no2.resample('D').mean().head()

Arriba se obtiene la media, pero así como funciona con `groupby`, pueden especificarse otros métodos:

In [ ]:
no2.resample('D').max().head()

El string para especificar la nueva frecuencia de tiempo: http://pandas.pydata.org/pandas-docs/dev/timeseries.html#offset-aliases  
Estos Strings pueden también combinarse con números. Por ejemplo: `'10D'`.

Más exploración de datos:

In [ ]:
no2.resample('M').mean().plot() # 'A'

In [ ]:
# no2['2012'].resample('D').plot()

In [ ]:
# %load snippets/01 - Introduccion95.py

<div class="alert alert-success">

<b>EJERCICIO</b>: La evolución de los promedios anuales, y el promedio de todas las estaciones:

 <ul>
  <li>Usar `resample` y `plot` para trazar los promedios anuales de cada estación.</li>
  <li>El promedio de todas las estaciones puede ser calculado tomando el medio de las diferentes columnas (`.mean(axis=1)`).</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion96.py

<div class="alert alert-success">

<b>EJERCICIO</b>: ¿Cómo se ve el *perfil mensual típico* para las diferentes estaciones?

 <ul>
  <li>Agregar una columna de 'month' al dataframe.</li>
  <li>Agrupar (groupby) por mes para obtener los promedios típicos mensuales de diferentes años.</li>
</ul>
</div>

Primero, agregamos una columna para indicar el mes (entero con un valor que va de 1 a 12):

In [ ]:
# %load snippets/01 - Introduccion97.py

Ahora, se calcula la media para cada mes durante diferentes años:

In [ ]:
# %load snippets/01 - Introduccion98.py

In [ ]:
# %load snippets/01 - Introduccion99.py

<div class="alert alert-success">

<b>EJERCICIO</b>: El perfil tipico diurno para diferentes estaciones:

 <ul>
  <li>Igual que para los meses, pero para la hora del día.</li>
</ul>
</div>

In [ ]:
# %load snippets/01 - Introduccion100.py

<div class="alert alert-success">

<b>EJERCICIO</b>: ¿Cuál es la diferencia entre el perfil típico diruno entre dias entre semana y fines de semana para la estación BASCH?

 <ul>
  <li>Agregar una columna `weekday` que defina los diferentes días de la semana.</li>
  <li>Agregar una columna `weekend` que defina si el día corresponde a fin de semana (días 5 o 6) o no (True/False).</li>
  <li>Pueden hacerse operaciones groupby sobre múltiples elementos al mismo tiempo. En este caso agrupamos por las dos columnas creadas y la hora del día.</li>
</ul>
</div>

Agregar la primera columna:

In [ ]:
no2.index.weekday?

In [ ]:
# %load snippets/01 - Introduccion102.py

Agregar la segunda columna:

In [ ]:
# %load snippets/01 - Introduccion103.py

Ahora podemos efectuar la operación groupby:

In [ ]:
# %load snippets/01 - Introduccion104.py

In [ ]:
# %load snippets/01 - Introduccion105.py

In [ ]:
# %load snippets/01 - Introduccion106.py

In [ ]:
# %load snippets/01 - Introduccion107.py

<div class="alert alert-success">

<b>EJERCICIO</b>: ¿Cuál es el número de excedenicas por hora arriba del límite europeo de 200 µg/m3?

Cuente el número de excedencias por hora arriba del límite para cada año y estación después del 2005. Construya un gráfico de barras para los conteos. Agregue una linea horizontal indicando el máximo de excedencias permitido por año (18)
<br><br>

Hints:

 <ul>
  <li>Cree un nuevo DataFrame llamado `exceedances`, (con valores booleanos) indicando si el umbral es excedido o no</li>
  <li>Recuerde que la suma total de valores "True" puede ser utilizada para contar elementos. Efectue este conteo usando groupby para cada año.</li>
  <li>Puede agregarse una linea horizontal utilizando la función `ax.axhline`.</li>
</ul>
</div>

In [ ]:
# re-reading the data to have a clean version
no2 = pd.read_csv('data/20000101_20161231-NO2.csv', sep=';', skiprows=[1], na_values=['n/d'], index_col=0, parse_dates=True)

In [ ]:
# %load snippets/01 - Introduccion109.py

In [ ]:
# %load snippets/01 - Introduccion110.py

In [ ]:
# %load snippets/01 - Introduccion111.py

# 9. Concatenación de Datos

- Concatenating data: `pd.concat`
- Merging and joining data: `pd.merge`
- Working with missing data: `isnull`, `dropna`, `interpolate`


## Más Documentación

* Documentación Oficial de Pandas: http://pandas.pydata.org/pandas-docs/stable/

* Libros

    * "Python for Data Analysis" por Wes McKinney
    * "Python Data Science Handbook" por Jake VanderPlas

* Tutoriales

  * https://github.com/jorisvandenbossche/pandas-tutorial
  * https://github.com/brandon-rhodes/pycon-pandas-tutorial

* Blog de Tom Augspurger

  * https://tomaugspurger.github.io/modern-1.html